前面基本上都是介绍的Zookeeper的使用场景以及如何使用，下面介绍最后一个问题，Zookeeper的原理。

# 1. Zookeeper的运行模式
Zookeeper包含两种运行模式：独立模式（单机模式，Standalone mode）、复制模式（replicated mode）。当然独立模式无法发挥Zookeeper的作用，只应用于测试环境或者开发环境中。不会应用于生产环境中，所以不做介绍了。

对于复制模式，至少需要3台服务器，并且需要有半数以上的服务器可用。举个例子：

- 我们有3台服务器，至少有2台是可用的，最多挂掉1台；
- 如果有4台服务器，至少有3台是可用的，最多挂掉1台；
- 如果有5台服务器，至少有3台是可用的，最多挂掉2台；
- 如果有6台服务器，至少有4台是可用的，最多挂掉2台；

上面可知4台的效果和3台都是最多能挂掉1台。出于这个原因，一个Zookeeper集群往往是奇数台（为了节省一台几乎没有起到任何作用的机器）。而Zookeeper每次更新数据都会要求有半数以上的服务器进行更新。（由于更新是原子性的，所以这半数的服务器上Zookeeper数据是完全相同的）

# 2. 原子广播
Zookeeper服务器角色包含两种：

- leader：一个领导者；
- follower：其他的所有服务器都作为跟随者；

当Zookeeper集群中有服务器收到写请求后，会将写请求转交给leader，然后leader会发送广播，要求所有的follower将该数据持久化，当有一半以上的服务器完成持久化之后，才会提交本次写请求。因此Zookeeper的文件系统的设计是原子性的，要不一起改，要不都不改（要是允许改部分，那么就会存在一个follower即存在最新的数据，也有可能丢失了一部分数据...）

而每一个写请求都会修改ZXID（事务ID），ZXID是递增的，他决定了服务器当前数据的更新状态。

当然对于这个问题，可能会存在一个困惑，那么如果此时我的客户端连接上了那些尚未持久化的服务器，那么我们岂不是读到了落后的数据了？针对这个问题，在客户端在向服务器读取数据的时候，服务器会强制对数据进行sync操作，sync操作会强制服务器进行数据更新。所以这时候就会保证数据是最新的了。

# 3. Zookeeper的领导者选举

参考：

- [zookeeper选举算法](https://blog.csdn.net/mweibiao/article/details/80916765)
- [理解zookeeper选举机制](https://www.cnblogs.com/ASPNET2008/p/6421571.html)

Zookeeper的服务器会分为两种角色：一个领导者（leader）、其他的所有都作为跟随者（follower）。只有领导者拥有向Zookeeper写入数据的权限。Zookeeper有一个领导者选举机制，当这个集群中没有领导者的时候（例如集群刚刚启动，或者领导者所在服务器挂掉了），那么集群中会发起投票，来选举出领导者。

集群中的节点在选举阶段会包含以下几种状态

- LOOKING，竞选状态。
- FOLLOWING，随从状态，同步leader状态，参与投票。
- OBSERVING，观察状态, 同步leader状态，不参与投票。
- LEADING，领导者状态。

投票选举有两个标准：

1. ZXID（事务ID：每次更新数据都会递增节点的事务ID，因此事务ID越大，表示他的数据越新）
2. myid（服务器ID：表示服务器编号）

然后集群会根据这两个标准发起一轮又一轮的投票，直到选择出一个领导者为止。投票过程为：

![image_1d6o22rpve491h3niotqk21ank9.png-388.7kB][1]

- 然后当集群启动的时候，服务器会查询是否存在领导者（有些服务器启动的慢，所以这时候可能已经选举出了领导者）
    - 如果存在领导者，则直接将自己变为跟随者。
    - 如果不存在领导者，则会发起投票推举自己作为领导者。（所以当领导者宕机的时候，所有的跟随者都会推举自己作为领导者。）
- 在发起投票后，服务器会接受其他服务器的信息，
    - 如果对方的ZXID大于自己的ZXID，则会将自己的票在下一轮投票的时候投给对方。
    - 如果对方的ZXID小于自己的ZXID，则会将自己的票在下一轮投票的时候投给自己。
    - 如果对方的ZXID等于自己的ZXID，但是myid大于自己的myid，那么在下一轮投票中会投给对方。
- 统计该轮票数，是否有服务器达到了一半服务器数以上的投票数
    - 如果是，则该服务器作为leader
    - 如果不是则发起下一轮投票。直到选举出leader未知

由于数据的写入必须要求一半以上的服务器完成才有效，因此选举出来的Leader的ZXID一定是最新的。

选举状态图：
![image_1d6o29ngn1oc4ue31oekq851f27m.png-290.7kB][2]
# 4. 会话

在我们的应用中，客户端会尝试连接Zookeeper服务器列表中的一台服务器，连接失败后会尝试另外一台，直到成功连接或者所有的连接都失败为止。

当连接上服务器之后会创建一个新的会话，会话会设置一个超时时间（由我们的应用决定）。Zookeeper会不断地通过客户端发送ping请求到Zookeeper服务器中来保证会话不过期。如果在超时时间之内没有ping通服务器，则会换其他的服务器接着ping，如果超时时间内ping通其他服务器，则该服务器会接管会话。如果达到了超时时间都无法连接上服务器，则会话失效，并且Zookeeper服务器会删除该会话创建的短暂的znode。

在Zookeeper安装的时候我们配置了`tickTime`参数，该参数就是设置ping的时间，而会话时长要求大于该值的2倍，小于该值的20倍，否则Zookeeper会将会话时长修改为2~20倍。通常`tickTime`会设置为2000ms。 

# 5. 总结

好了Zookeeper的基础使用方式及原理就介绍到这里，如果以后有机会使用，再来介绍一些其他的东西。


  [1]: ../imgs/image_1d6o22rpve491h3niotqk21ank9.png
  [2]: ../imgs/image_1d6o29ngn1oc4ue31oekq851f27m.png